In [1]:
import sys
sys.path.append("/storage/BonnieH/selection_project/helper_functions")
from ABC_functions import *

In [2]:
# Code to write example loci to file:

def WriteExampleLoci(inFile, outFile):
    inFile = '/storage/BonnieH/selection_project/ssc_files/allele_freqs/merged_filtered.txt'
    outFile = '/storage/BonnieH/selection_project/ssc_files/allele_freqs/example_loci.txt'

    allele_freqs_file = open(inFile, 'r')
    results = open(outFile, 'w')
    
    codis_list = [('chr2', 1489651), ('chr5', 123775553), ('chr5', 150076322), ('chr7', 84160225), ('chr8', 124894863), \
                  ('chr11', 2171086), ('chr13', 82148025), ('chr16', 86352702), ('chr18', 63281667)]
        
    disease_list = [('chr19', 45770205), ('chr19', 13207859), ('chr13', 70139384), ('chr5', 146878729)]
    
    early_onset_list = [('chr6', 45315923), ('chr6', 45362057), ('chr6', 45474374), ('chr6', 45478639), \
                       ('chr6', 45506657), ('chr6', 45515702), ('chr6', 45523256), ('chr2', 176089200), \
                       ('chr2', 176094386)]
                  
    codisFile = '/storage/BonnieH/selection_project/annotations/hg38_codis.bed'
    diseaseFile = '/storage/BonnieH/selection_project/annotations/hg38_disease.txt'
    earlyonsetFile = '/storage/BonnieH/selection_project/annotations/hg38_earlyonset.txt'

    codis = open(codisFile, 'r')
    codis_dic = {}
    for line in codis:
        info = line.strip().split('\t')
        chrom = info[0]
        start = int(info[1])
        end = int(info[2])
        name = info[5]
        codis_dic[(chrom, start)] = name
    codis.close()
        
    disease = open(diseaseFile, 'r')
    disease_dic = {}
    for line in disease:
        info = line.strip().split('\t')
        chrom = info[0]
        start = int(info[1])
        end = int(info[2])
        name = info[5]
        disease_dic[(chrom, start)] = name
    disease.close()

    early = open(earlyonsetFile, 'r')
    early_dic = {}
    for line in early:
        info = line.strip().split('\t')
        chrom = info[0]
        start = int(info[1])
        end = int(info[2])
        name = info[4]
        early_dic[(chrom, start)] = name
    early.close()

    Lines = allele_freqs_file.readlines()
    
    for line in Lines:
        
        # Get information from line
        info = line.strip().split('\t')
        chrom = info[0]
        start = int(info[1])
        end = int(info[2])
        per = int(info[3])
        freqs = info[4]
            
        # Get info about reference allele (length in base pairs, repeat units)
        ref_length_bp = end - start + 1
        ref_length_ru = int(ref_length_bp/per)
            
        tup = (chrom, start)
            
            
        if (tup in disease_dic) or (tup in early_dic) or (tup in codis_dic):
            print(tup)
            results.write(line)
            '''
            elif per == 2 and ref_length_ru >= 11:
                results.write(line)
                
            elif per == 3 and ref_length_ru >= 5:
                results.write(line)
                
            elif per == 4 and ref_length_ru >= 7:
                results.write(line)
            '''
    results.close()        
    allele_freqs_file.close()
    print("Done")

In [3]:
# Code to filter out short loci based on optimal allele length threshold 

def FilterShortLoci(inFile, outFile):
    
    allele_freqs_file = open(inFile, 'r')
    results = open(outFile, 'w')
    i = 0
    j = 0
    k = 0
    
    for line in allele_freqs_file:
        j = j + 1
        
        # Get information from line
        info = line.strip().split('\t')
        chrom = info[0]
        start = int(info[1])
        end = int(info[2])
        freqs = info[3]
        per = int(info[4])
        if freqs == '.':
            print(line)
        
        #  Only continue if freqs != ''
        if freqs != '.' : 
            
            # Get info about reference allele (length in base pairs, repeat units)
            ref_length_bp = end - start + 1
            ref_length_ru = int(ref_length_bp/per)
            
            optimal_ru = Process_Freqs(freqs, per, end, start, False)
            
            if ref_length_ru != optimal_ru:
                i = i + 1
                #print((chrom, start))
            if per == 1 and optimal_ru >= 22:
                results.write(line)
                k = k + 1
                
            elif per == 2 and optimal_ru >= 11:
                results.write(line)
                k = k + 1 
                
            elif per == 3 and optimal_ru >= 5:
                results.write(line)
                k = k + 1
                
            elif per == 4 and optimal_ru >= 7:
                results.write(line)
                k = k + 1
                
            # Adding RUNX2 tracts?
            #elif chrom == "chr6" and start == 45442365:
             #   results.write(line)
                
            #elif chrom == "chr6" and start == 45457637:
             #   results.write(line)
            
    results.close()        
    allele_freqs_file.close()
    print("Done")
    print(i)
    print(j)
    print(k)

In [4]:
def main():
    inFile = '/storage/BonnieH/selection_project/ssc_files/allele_freqs/allele_freqs_per_1234.txt'
    outFile = '/storage/BonnieH/selection_project/ssc_files/allele_freqs/allele_freqs_filt_per_1234.txt'
    FilterShortLoci(inFile, outFile)

In [5]:
if __name__ == '__main__':
    main()


1	32642768	32642788	.	1	A	0.0	0.0	0.0	0.0	0.0	1.0	RF00019

1	41369862	41369873	.	4	AGGG	0.0	0.0	0.0	0.0	0.0	1.0	AC093151.2

1	42110162	42110217	.	2	AT	0.0	0.0	0.0	0.0	0.0	1.0	AC114492.1

1	51580893	51580954	.	2	AT	0.0	0.0	0.0	0.0	0.0	1.0	CALR4P

1	54523412	54523443	.	4	AAAC	0.0	0.0	0.0	0.0	0.0	1.0	AC099796.2

1	61925200	61925268	.	1	A	0.0	0.0	0.0	0.0	0.0	1.0	PATJ

1	71275648	71275694	.	1	A	0.0	0.0	0.0	0.0	0.0	1.0	ZRANB2-AS2

1	71793264	71793279	.	2	AT	0.0	0.0	0.0	0.0	0.0	1.0	NEGR1-IT1

1	112449728	112449753	.	2	AC	0.0	0.0	0.0	0.0	0.0	1.0	CTTNBP2NL

1	120402335	120402373	.	1	A	0.0	0.0	0.0	0.0	0.0	1.0	NBPF8

1	150719256	150719294	.	1	A	0.0	0.0	0.0	0.0	0.0	1.0	HORMAD1

1	168471925	168471949	.	1	A	0.0	0.0	0.0	0.0	0.0	1.0	QRSL1P1

1	220576005	220576016	.	4	AGGG	0.0	1.0	0.0	0.0	0.0	0.0	RN7SL464P

1	228679066	228679113	.	1	A	0.0	0.0	0.0	0.0	0.0	1.0	FTH1P2

2	76070934	76070945	.	2	AT	0.0	0.0	0.0	0.0	0.0	1.0	SUCLA2P2

2	83341795	83341806	.	4	AATT	0.0	0.0	0.0	0.0	0.0	1.0	AC138623.1

2	92270611	9

19	39795728	39795760	.	1	A	0.0	0.0	0.0	0.0	0.0	1.0	LEUTX

19	42663235	42663730	.	2	AT	0.0	0.0	0.0	0.0	0.0	1.0	RPS10P28

19	46453128	46453178	.	1	A	0.0	0.0	0.0	0.0	0.0	1.0	PNMA8A

19	50827951	50827965	.	3	AAC	0.0	0.0	0.0	0.0	0.0	1.0	KLK15

20	17122804	17122847	.	1	A	0.0	0.0	0.0	0.0	0.0	1.0	RNU6-27P

20	27871304	27871317	.	1	A	0.0	0.0	0.0	0.0	0.0	1.0	FRG1CP

20	29129667	29129685	.	1	A	0.0	0.0	0.0	0.0	0.0	1.0	FRG1DP

20	29141112	29141124	.	1	A	0.0	0.0	0.0	0.0	0.0	1.0	FRG1DP

20	29556088	29556099	.	4	AATG	0.0	0.0	0.0	0.0	0.0	1.0	ABBA01031666.1

20	32311701	32311732	.	1	A	0.0	0.0	0.0	0.0	0.0	1.0	KIF3B

20	36642168	36642197	.	1	A	0.0	0.0	0.0	0.0	0.0	1.0	SLA2

20	37384667	37384678	.	3	AAG	0.0	1.0	0.0	0.0	0.0	0.0	SRC

20	37725537	37725548	.	4	AGGC	0.0	0.0	0.0	0.0	0.0	1.0	CTNNBL1

20	42556549	42556562	.	2	AT	0.0	0.0	0.0	0.0	0.0	1.0	AL035666.1

20	51740566	51740574	.	3	AAC	0.0	0.0	0.0	0.0	0.0	1.0	AL138807.1

20	62491854	62491865	.	4	ACCC	0.0	0.0	0.0	0.0	0.0	1.0	AL499627.2

21	5448941	5448955	.	3